In [1]:
import numpy as np
import pandas as pd
import math
import copy

In [2]:
rating_df_columns = ["UserId", "MovieId", "Rating", "TimeStamp"]
rating_df = pd.read_table("ml-1m/ratings.dat", sep="::", names=rating_df_columns)

/home/vedant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
rating_df.drop(index =range(100000,1000209), inplace=True)

In [4]:
rating_matrix = np.asarray(rating_df.pivot(index = "UserId", columns = "MovieId", values = "Rating").fillna(0))

In [5]:
### Mean center

def mean_center(A):
    mean_centered_A = np.copy(A)
    for row in range(A.shape[0]):
        ratings_sum = A[row].sum()
        movies_rated = (A[row] > 0).sum()
        mean = ratings_sum/movies_rated
        for i in A[row].nonzero():
            mean_centered_A[row][i] -= mean
    return mean_centered_A

In [6]:
rating_matrix_centered = mean_center(rating_matrix)

In [7]:
### SVD
# A = np.asarray([[3,1,1],[-1,3,1]])

In [8]:
A = rating_matrix_centered

In [9]:
AAt = np.matmul(A, A.T)
AtA = np.matmul(A.T, A)

In [10]:
eigenvalue_AAt, eigenvector_AAt = np.linalg.eig(AAt)
eigenvalue_AtA, eigenvector_AtA = np.linalg.eig(AtA)

In [11]:
argsort_eigenval_AAt = np.argsort(-eigenvalue_AAt)
argsort_eigenval_AtA = np.argsort(-eigenvalue_AtA)

In [12]:
U = np.zeros(eigenvector_AAt.shape)
sigma1 = np.sqrt(np.sort(eigenvalue_AAt))[::-1]
# for i,j in enumerate(argsort_eigenval_AAt):
#     U[:,i] = eigenvector_AAt[:,j]

In [13]:
V = np.zeros(eigenvector_AtA.shape)
sigma2 = np.sqrt(np.sort(eigenvalue_AtA))[::-1]
for i,j in enumerate(argsort_eigenval_AtA):
    V[:,i] = eigenvector_AtA[:,j]

/home/vedant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


In [14]:
sigma = np.zeros(A.shape)
if sigma1.shape[0] < sigma2.shape[0]:
    sigma[:, :A.shape[0]] = np.diag(sigma1)
else:
    sigma[:A.shape[1], :] = np.diag(sigma2)

In [15]:
for i in range(U.shape[1]):
    U[:,i] = (A.dot(V[:,i]))/sigma[i][i]

In [16]:
n_component = 10
sigma_reduced = sigma[:,:]
V_reduced = V[:,:]
A_pred = U.dot(sigma_reduced.dot(V_reduced.T))


In [17]:
# u,s,vh = np.linalg.svd(A)
# S = np.zeros(A.shape)
# S[:A.shape[0], :A.shape[0]] = np.diag(s)
# Apred_svd = u.dot(S.dot(Vh))
# S = S[:,:]
# Vh = vh[:,:]

In [19]:
error = 0
cnt = 0
for i in range(rating_matrix_centered.shape[0]):
    for j in range(rating_matrix_centered.shape[1]):
        if rating_matrix[i][j] != 0:
            error += ((rating_matrix_centered[i][j] - A_pred[i][j]) ** 2)
            cnt += 1
mse = np.sqrt(error/cnt)
print(mse)

6.022876915127478e-14
